In [1]:
!pip install numpy

# Lista de frameworks e bibliotecas

$$ \frac {x_1}{x_2} $$

In [3]:
import numpy as np

In [4]:
np.version

<module 'numpy.version' from 'D:\\Developer\\anaconda3\\lib\\site-packages\\numpy\\version.py'>

In [3]:
import pickle as pickle

pb = pickle.PickleBuffer(b"foo")
data = pickle.dumps(pb, protocol=5)
assert pickle.loads(data) == b"foo"